In [185]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime

In [186]:
def selecionar_aprovados(dados):

    df = dados.copy(deep=True)

    # Seleciona os 81 primeiros candidatos aprovados
    df.loc[df.loc[df["APROVADO DISCURSIVA"] == True].head(81).index, 'SITUAÇÃO PARCIAL'] = "Aprovado nas vagas de ampla concorrência"

    # Seleciona os candidatos aprovados PCD que ainda não foram incluídos
     
    df.loc[df.loc[(df['SITUAÇÃO PARCIAL'].isna()) & 
                  (df["SITUACAO OBJETIVA"] == "Aprovado PcD") &
                  (df["APROVADO DISCURSIVA"] == True)].head(9).index, 'SITUAÇÃO PARCIAL'] = "Aprovado nas vagas de cota PCD"

    # Caso não haja 9 candidatos aprovados PCD, completa com aprovados na Ampla
    if len(df[df['SITUAÇÃO PARCIAL'] == "Cota PCD"]) < 9:
        df.loc[df.loc[(df['SITUAÇÃO PARCIAL'].isna()) &
                      (df["APROVADO DISCURSIVA"] == True)].head(9 - len(df[df['SITUAÇÃO PARCIAL'] == "Aprovado nas vagas de cota PCD"])).index, 'SITUAÇÃO PARCIAL'] = "Aprovado nas vagas de ampla concorrência"

    df.loc[(df['SITUAÇÃO PARCIAL'].isna()) & 
           (df["APROVADO DISCURSIVA"] == True), 'SITUAÇÃO PARCIAL'] = "Cadastro de Reservas"

    df['SITUAÇÃO PARCIAL'] = df['SITUAÇÃO PARCIAL'].fillna('Reprovado na discursiva')
      
    # Retorna os 81 primeiros aprovados + os 9 primeiros aprovados PCD
    return df

In [187]:
resultado_objetiva = pd.read_csv('./data/resultado-objetiva.csv', sep=";", index_col=['INSC_NUMB', 'NOME'])
resultado_discursiva = pd.read_csv('./data/resultado-discursiva.csv', sep=";", index_col=['INSC_NUMB', 'NOME'])

In [188]:
resultado_preliminar = resultado_objetiva.merge(resultado_discursiva, on=['INSC_NUMB', 'NOME'], how='left')

In [189]:
resultado_preliminar['TOTAL'] = resultado_preliminar['TOTAL OBJETIVA'] + resultado_preliminar['TOTAL DISCURSIVA'].fillna(value=0)
resultado_preliminar['APROVADO DISCURSIVA'] = resultado_preliminar['TOTAL DISCURSIVA'] >= 40
resultado_preliminar['NASC'] = pd.to_datetime(resultado_preliminar['NASC'], format='%d/%m/%Y')
today = datetime.today()
resultado_preliminar['IDADE'] = (today - resultado_preliminar['NASC']).astype('timedelta64[Y]')
resultado_preliminar['IDOSO'] = resultado_preliminar['IDADE'] >= 60

In [190]:
resultado_preliminar_ordenado = resultado_preliminar.sort_values(by=['APROVADO DISCURSIVA', 'TOTAL', 'IDOSO', 'TOTAL DISCURSIVA', 'P2', 'CD', 'DEV', 'BD', 'NASC'], 
                                               ascending=[False, False, False, False, False, False, False, False, True])

In [191]:
resultado_preliminar_ordenado = resultado_preliminar_ordenado.reset_index(drop=False)
resultado_preliminar_ordenado['POS'] = resultado_preliminar_ordenado.index + 1

In [ ]:
resultado_preliminar_ordenado[resultado_preliminar_ordenado["SITUACAO OBJETIVA"] == "Aprovado PcD"]

In [193]:
resultado_preliminar_ordenado = selecionar_aprovados(resultado_preliminar_ordenado)

In [ ]:
resultado_preliminar_ordenado[['POS', 'INSC_NUMB', 'NOME', 'TOTAL', 'IDOSO', 'TOTAL DISCURSIVA', 'P2', 'CD','DEV', 'BD', 'NASC', 'VAGAS']]